**Rotator Performance Turning**

This Jupyter notebook is used to turn the controller parameters by Tekniker.
The test will be performed on summit with the real hardware.
Before that, the Tucson test stand is used to develop the test procedure.
The rotator commandable SAL component (CSC) is used to control the rotator hardware.
The published event and telemetry are stored in the engineering facility database (EFD).

It is noted that the **MTMount** telemetry data is required.
Otherwise, the rotator CSC will put the low-level controller into the **Fault** state.

The **setup** notebook can be used to bring up rotator and mount/CCW.

**Import Modules**

In [1]:
%matplotlib inline
%matplotlib widget
import asyncio
from datetime import datetime
import time
from astropy.time import Time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from lsst.ts import salobj
from lsst.ts import simactuators
from lsst_efd_client import EfdClient

**Custom Functions**

In [2]:
def retrieve_efd_client(is_summit=True):
    """
    Retrieves a client to EFD.

    Parameters
    ----------
    is_summit : bool, optional
        This notebook is running on the summit or not. If not, the returned object will point
        to the test stand at Tucson.

    Returns
    -------
    EfdClient : The interface object between the Nublado and summit/Tucson EFD.
    """
    efd_name = "summit_efd" if is_summit else "tucson_teststand_efd"        
    return EfdClient(efd_name)

In [3]:
def script_log_start(controller_script, test_title, test_detail):
    """Log the start of test case in script controller.

    Parameters
    ----------
    controller_script : lsst.ts.salobj.controller.Controller
        Script controller.
    test_title : str
        Test title.
    test_detail : str
        Test detail.

    Returns
    -------
    astropy.time.core.Time
        Time object.
    """
    time_now = datetime.now()
    controller_script.log.info(f"START -- {test_title} -- LVV-T2539 -- {test_detail} -- Starting Time: {time_now} UTC")
    return Time(time_now.isoformat(), scale="utc", format="isot")

In [4]:
def script_log_stop(controller_script, test_title, test_detail):
    """Log the stop of test case in script controller.

    Parameters
    ----------
    controller_script : lsst.ts.salobj.controller.Controller
        Script controller.
    test_title : str
        Test title.
    test_detail : str
        Test detail.

    Returns
    -------
    astropy.time.core.Time
        Time object.
    """
    time_now = datetime.now()
    controller_script.log.info(f"STOP -- {test_title} -- LVV-T2539 -- {test_detail} -- Stopping Time: {time_now} UTC")
    return Time(time_now.isoformat(), scale="utc", format="isot")

In [5]:
async def track_targets(csc):
    """Track the targets.

    Parameters
    ----------
    csc : lsst.ts.salobj.remote.Remote
        Remote object of the rotator CSC.
    """

    lasting_time = 30
    velocities = [0.0042, 0.01, 0.015, 0.02, 0.025, 0.03, 0.035, 0.04, 0.045, 0.05, 0.055, 0.06, 0.065, 0.07, 0.075, 0.08]

    for sign in (1, -1):
        for velocity in velocities:
            await track_target_time(csc, 0, sign * velocity, lasting_time)
            await make_sure_csc_enabled(csc)
            await move_origin(csc)

        for velocity in velocities:
            await track_target_time(csc, sign * 10, sign * velocity, lasting_time)
            await make_sure_csc_enabled(csc)
            await move_origin(csc)

In [6]:
async def track_target_time(csc, start_position, velocity, lasting_time):
    """Track the target with a lasting time.

    Parameters
    ----------
    csc : lsst.ts.salobj.remote.Remote
        Remote object of the rotator CSC.
    start_position : float
        Starting position of ramp (deg).
    velocity : float
        Velocity of motion along the ramp (deg/sec).
    lasting_time : float or int
        Lasting time of tracking target. This value should be >= 0.
    """

    end_position = start_position + velocity * lasting_time
    await track_target(csc, start_position, end_position, velocity)

In [7]:
async def track_target(csc, start_position, end_position, velocity):
    """Track the target.

    Parameters
    ----------
    csc : lsst.ts.salobj.remote.Remote
        Remote object of the rotator CSC.
    start_position : float
        Starting position of ramp (deg).
    end_position : float
        Ending position of ramp (deg).
    velocity : float
        Velocity of motion along the ramp (deg/sec).
    """    

    await make_sure_csc_enabled(csc)
    
    # 20 Hz is 0.05 ms
    track_advance_time = 0.05
    ramp_generator = simactuators.RampGenerator(
        [start_position], [end_position], [velocity], track_advance_time)

    await csc.cmd_trackStart.start(timeout=15)
    await asyncio.sleep(0.15)
    for positions, velocities, tai in ramp_generator():
        await csc.cmd_track.set_start(
            angle=positions[0],
            velocity=velocities[0],
            tai=tai,
            timeout=5,
        )
        await asyncio.sleep(0.15)

    await asyncio.sleep(1)

In [8]:
async def move_origin(csc):
    """Move to the origin.

    Parameters
    ----------
    csc : lsst.ts.salobj.remote.Remote
        Remote object of the rotator CSC.
    """

    sleep_time = 3
    await asyncio.sleep(sleep_time)

    await make_sure_csc_enabled(csc)

    # Workaround the problem of MOOG state machine to decide a new point-to-point movement
    await csc.cmd_move.set_start(position=0.1, timeout=90)
    await asyncio.sleep(sleep_time)

    await make_sure_csc_enabled(csc)

    await csc.cmd_move.set_start(position=0, timeout=90)
    await asyncio.sleep(sleep_time)

In [9]:
async def make_sure_csc_enabled(csc):
    """Make sure the CSC is enabled.

    Parameters
    ----------
    csc : lsst.ts.salobj.remote.Remote
        Remote object of the rotator CSC.
    """
    await asyncio.sleep(3)
    data = csc.evt_summaryState.get()
    await asyncio.sleep(3)
    if data.summaryState != salobj.State.ENABLED:
        await salobj.set_summary_state(csc, salobj.State.ENABLED, timeout=90)

**Instantiate the EFD and CSC**

Need to make sure **ospl** is running for the data distribution system (DDS) communication.

In [10]:
is_summit = True
efd_client = retrieve_efd_client(is_summit=is_summit)
domain = salobj.Domain()
rotator = salobj.Remote(domain, "MTRotator")
await rotator.start_task
await rotator.cmd_setLogLevel.set_start(level=10)

**MTMount** controller is used to run the camera cable wrapper (CCW).

In [11]:
mtmount = salobj.Remote(domain, "MTMount")
await mtmount.start_task

state_mtmount = await mtmount.evt_summaryState.aget(timeout=10)
if state_mtmount != salobj.State.ENABLED:
    await salobj.set_summary_state(mtmount, salobj.State.ENABLED, timeout=10)

await mtmount.cmd_enableCameraCableWrapFollowing.set_start()

Set the rotator CSC to be **Enabled** state.

In [12]:
await salobj.set_summary_state(rotator, salobj.State.ENABLED, timeout=10)

[<State.FAULT: 3>, <State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

In [13]:
await move_origin(rotator)

**Setup the Script**

In [17]:
time_start_evaluation = datetime.now()
test_title = "Evaluate the Rotator Controller Performance"
script = salobj.Controller("Script", index=10)

# Wait 10 second may help with DDS problems; closing all other kernels may help too
await asyncio.sleep(10)
print(f"Time to start the evaluation is {time_start_evaluation}.")

Time to start the evaluation is 2022-06-13 13:48:47.083775.


**Baseline**

In [24]:
time_start = script_log_start(script, test_title, "Baseline")

In [ ]:
await track_targets(rotator)

In [21]:
time_stop = script_log_stop(script, test_title, "Baseline")

**Second Time**

In [ ]:
time_start = script_log_start(script, test_title, "Baseline 2")

In [ ]:
await track_targets(rotator)

In [ ]:
time_stop = script_log_stop(script, test_title, "Baseline 2")

**Third Time**

In [ ]:
time_start = script_log_start(script, test_title, "Baseline 3")

In [ ]:
await track_targets(rotator)

In [ ]:
time_stop = script_log_stop(script, test_title, "Baseline 3")